In [2]:
from plyfile import PlyData
import jax 
import jax.numpy as jnp 
from jax import grad, jacfwd
import polyscope as ps

from jaxlie import SO3, SE3

import numpy as np
import open3d as o3d 
import trimesh as tm

import meshcat
from meshcat import Visualizer
import meshcat.geometry as mc_geom
import meshcat.transformations as mc_trans

import time

objc[20750]: Class GLFWWindowDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x12f646e90) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x16c5bedb0). One of the two will be used. Which one is undefined.
objc[20750]: Class GLFWApplicationDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x12f646e68) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x16c5bee28). One of the two will be used. Which one is undefined.
objc[20750]: Class GLFWContentView is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x12f646ee0) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x16c5bee50). One of the two will be used. Which one is undefin

In [22]:
v2 = np.array([0., 1., 1.])
v2 = v2/np.linalg.norm(v2)
v1 = np.array([0., 0., 1.])
def loss(params):
    w = params['w']
    v = params['v']
    _R = SO3.exp(w)
    return jnp.sum(jnp.dot(v2, _R@v1))

In [27]:
v2

array([0.        , 0.70710678, 0.70710678])

In [28]:
SE3.from_rotation_and_translation(_R, v2)@v1

Array([ 8.4449729e-04, -3.4630299e-05,  3.4987926e-05], dtype=float32)

In [16]:
vis = Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [18]:
body1 = vis['v1']
body1.set_object(mc_geom.triad(scale=0.5))


In [ ]:
w = jnp.array([0.1,0.2,0.3])
v = jnp.array([.0,0.,.2])

params = {'w': w, 'v':v}
for _ in range(100):
    _grad = grad(loss)(params)
    params['w'] = params['w'] - 0.1*_grad['w']
    params['v'] = params['v'] - 0.1*_grad['v']
    _R = SO3.exp(params['w'])
    _T = SE3.from_rotation_and_translation(_R, v2).as_matrix()
    body1.set_transform(np.array(_T, dtype=np.float64))
    print(params, loss(params))
    time.sleep(0.1)

In [1]:
dataset = o3d.data.BunnyMesh()
plydata = PlyData.read(dataset.path)
vertices = np.vstack((
    plydata['vertex']['x'],
    plydata['vertex']['y'],
    plydata['vertex']['z']
)).T

# _min_pnt = vertices.min(axis=0)
# _max_pnt = vertices.max(axis=0)
# _mid_pnt = vertices.mean(axis=0)
# vertices = (vertices - _mid_pnt)/(_max_pnt-_min_pnt)

faces = np.array(np.vstack(plydata['face']['vertex_indices']))

mesh = tm.Trimesh(vertices=vertices, faces=faces)

ds_mesh = mesh.simplify_quadric_decimation(int(mesh.vertices.shape[0]/10))

NameError: name 'o3d' is not defined

In [60]:
ps.init()

ps_mesh = ps.register_surface_mesh("my mesh", ds_mesh.vertices, ds_mesh.faces)
ps_mesh.add_vector_quantity("rand vecs", ds_mesh.vertex_normals, enabled=True)


ps.show()